In [1]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

import pandas as pd
import numpy as np
import openpyxl
import json
import re
import ast
from collections import Counter
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF, BNode, RDFS

import rdflib
from rdflib import Literal, RDF, URIRef, Namespace
from rdflib.namespace import RDFS,RDF,XSD
from tqdm import tqdm
import dateparser
import dateutil.parser as date_parse
data_dir = "../data_/"

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


## 1.  Player

In [2]:
player_df  = pd.read_csv(data_dir + "player_join.csv", index_col = 0)
player_df.head()

,team_history,player_url_x,name,Born,Placeofbirth,Nationality_x,Height_x,Weight,Position(s),Foot,...,Height_y,Nationality_y,Positions,Strengths,Weaknesses,Style_of_Play,Current Team,DOB,birth_block_y,player_id_y
0,"{'0': {'period': '07/2011 - 06/2012', 'team': ...",https://www.worldfootball.net/player_summary/m...,Michael Johnson,['24.02.1988'],"['Urmston', 'England']",['England'],['183cm'],['79kg'],['DefensiveMidfielder'],['right'],...,183cm,England,Midfielder,NaN,NaN,NaN,NaN,24-02-1988,19880224.0,2000534.0
1,"{'0': {'period': '10/2017 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/r...,Roger Johnson,['28.04.1983'],"['Ashford', 'England']",['England'],['191cm'],['78kg'],['CentreBack'],['right'],...,191cm,England,Defender (Centre),['Blocking the ball'],['Passing'],"['Indirect set-piece threat', 'Likes to play l...",NaN,28-04-1983,19830428.0,2000086.0
2,"{'1': {'period': '07/2020 - 08/2020', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Jervis,['17.09.1991'],"['Wolverhampton', 'England']",['England'],['191cm'],NaN,"['RightWinger', 'CentreForward']",['right'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'1': {'period': '07/2016 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Bidwell,['21.03.1993'],"['Southport', 'England']",['England'],['183cm'],['80kg'],"['LeftBack', 'LeftMidfielder']",['left'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'0': {'period': '07/2013 - 06/2016', 'team': ...",https://www.worldfootball.net/player_summary/b...,Bosingwa,['24.08.1982'],"['Mbandaka', 'CongoDR']","['Portugal', 'CongoDR']",['183cm'],['75kg'],"['RightBack', 'RightMidfielder']",['right'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data preprocessing

In [3]:
## Position(s)
def f(x):
    if type(x) == float:
        return x
    else:
        x = ast.literal_eval(x)
        res = [" ".join([s for s in re.split("([A-Z][^A-Z]*)", i) if s]) for i in x]
        return res
    
player_df["Position(s)"] = player_df["Position(s)"].apply(f)

In [4]:
## Positions
def f(x):
    if type(x) == float:
        return x
    tmp = x.split('(')
    if len(tmp) == 1:
        return tmp
    if len(tmp) == 2:
        res = []
        sub = tmp[1]
        sub = re.sub("\(", "" , sub)
        sub = re.sub("\)", "" , sub)
        sub = re.sub(",,",",",sub)
        sub = sub.split(",")
        res = [i +" "+ tmp[0] for i in sub]
        res = [i.strip() for i in res]
        return res
    
def g(x):
    if type(x) == float:
        return x
    elif x[0] == 'C Defensive Midfielder':
        x[0] = 'Centre Defensive Midfielder'
    return x
    
    
player_df["Positions"] = player_df["Positions"].apply(f).apply(g)

## KG

In [5]:
QB = Namespace('http://purl.org/linked-data/cube#')
MYNS = Namespace('http://dsci558.org/myprojectnamespace#')
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
SCHEMA = Namespace("https://schema.org/")
WIKIDATA = Namespace("https://wikidata.org/")

In [6]:
my_kg = rdflib.Graph()
my_kg.bind('qb',QB)
my_kg.bind('my_ns', MYNS)
my_kg.bind('rdfs',RDFS)
my_kg.bind("schema", SCHEMA)
my_kg.bind('rdfs',RDFS)
my_kg.bind('wiki', WIKIDATA)

In [7]:
x = [ast.literal_eval(i) for i in list(player_df["Nationality_x"].dropna())]
flatten_list = [j for sub in x for j in sub]

In [8]:
country_list_y = list(set(player_df["Nationality_y"].dropna()))

In [9]:
## Country Entities
x = [ast.literal_eval(i) for i in list(player_df["Nationality_x"].dropna())]
flatten_list = [j for sub in x for j in sub]
country_list = set(flatten_list + country_list_y)

for country in country_list:
    revised_name = re.sub("[^a-zA-Z]", "_", country)
    country_uri = URIRef(MYNS[revised_name])
    my_kg.add((country_uri, RDF.type, SCHEMA.Country))
    my_kg.add((country_uri, RDFS.label, Literal(country)))

In [10]:
city_list = set(player_df["Placeofbirth"].dropna().apply(lambda x : ast.literal_eval(x)[0]))

In [11]:
## City Entities
for city in city_list:
    revised_name = re.sub("[^a-zA-Z]", "_", city)
    city_uri = URIRef(MYNS[revised_name])
    my_kg.add((city_uri, RDF.type, MYNS['city']))
    my_kg.add((city_uri, RDF.type, SCHEMA.Place))
    my_kg.add((city_uri, RDFS.label, Literal(city)))

In [12]:
tmp = []
for i in list(player_df["Positions"].dropna()):
    tmp += i
for i in list(player_df["Position(s)"].dropna()):
    tmp += i
position_list = set(tmp)

In [13]:
## Position Entities
for position in position_list:
    revised_name = re.sub("[^a-zA-Z]", "_", position)
    position_uri = URIRef(MYNS[revised_name])
    my_kg.add((position_uri, RDF.type, MYNS['position']))
    my_kg.add((position_uri, RDFS.label, Literal(position)))

In [14]:
## Team Entities

In [15]:
player_df = player_df.fillna("nan")

In [16]:
def f(x):
    if x.player_id_x != 'nan':
        tmp1 = str(int(x.player_id_x))
    else:
        tmp1 = "nan"
    if x.player_id_y != 'nan':
        tmp2 = str(int(x.player_id_y))
    else:
        tmp2 = "nan"
    return tmp1 + "_" + tmp2
player_df["player_id_tot"] = player_df.apply(f, axis = 1 )

In [17]:
player_df["team_history"][0]

"{'0': {'period': '07/2011 - 06/2012', 'team': 'Leicester City', 'position': 'Midfielder', 'team_url': 'https://www.worldfootball.net/teams/leicester-city/'}, '1': {'period': '07/2006 - 07/2011', 'team': 'Manchester City', 'position': 'Midfielder', 'team_url': 'https://www.worldfootball.net/teams/manchester-city/'}}"

In [18]:
json_list = []
for js in list(player_df[player_df["team_history"]!='nan']["team_history"]):
    js = re.sub("L'A", "LA", js)
    js = re.sub("l'A", "lA", js)
    js = re.sub("O'H", "OH", js)
    js = re.sub("e'e", "ee", js)
    js = re.sub("d'A", "dA", js)
    js = re.sub("a'a", "aa", js)
    js = re.sub("L'H", "LH", js)
    js = re.sub("d'Á", "dÁ", js)
    js = re.sub("'s ", "s ", js)
    
    js = re.sub("'", "\"", js)
    json_list.append(json.loads(js))

In [19]:
json_list[0]

{'0': {'period': '07/2011 - 06/2012',
  'team': 'Leicester City',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/leicester-city/'},
 '1': {'period': '07/2006 - 07/2011',
  'team': 'Manchester City',
  'position': 'Midfielder',
  'team_url': 'https://www.worldfootball.net/teams/manchester-city/'}}

In [20]:
team_list = []
for js in json_list:
    jss = js.values()
    for i in jss:
        team = i["team"]
        team_url = i["team_url"]
        if team != "":
            team_list.append((team, team_url))
    

In [21]:
for team in set(team_list):
    revised_name = re.sub("[^a-zA-Z]", "_", team[0])
    team_uri = URIRef(MYNS[revised_name])
    my_kg.add((team_uri, RDF.type, SCHEMA.SportsTeam))
    my_kg.add((team_uri, RDFS.label, Literal(team[0])))
    my_kg.add((team_uri, MYNS["worldfootball_url"], Literal(team[1])))
    

In [22]:
json.loads(re.sub("'", "\"", player_df["team_history"].dropna()[9]))

{'0': {'period': '01/2018 - 12/2019',
  'team': 'Balmazújvárosi FC',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/balmazujvarosi-fc/'},
 '1': {'period': '02/2016 - 06/2017',
  'team': 'Nyíregyháza Spartacus',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/nyiregyhaza-spartacus/'},
 '2': {'period': '07/2015 - 02/2016',
  'team': 'Mezőkövesdi SE',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/mezoekoevesdi-se/'},
 '3': {'period': '07/2014 - 06/2015',
  'team': 'Puskás FC',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/puskas-fc/'},
 '4': {'period': '08/2013 - 06/2014',
  'team': 'Mezőkövesdi SE',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/mezoekoevesdi-se/'},
 '5': {'period': '09/2012 - 08/2013',
  'team': 'Spartak Myjava',
  'position': 'Forward',
  'team_url': 'https://www.worldfootball.net/teams/spartak-myjava/'},
 '6': {'period': '01

## Predicate

In [23]:
predicate_uri = MYNS["position"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("position", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [24]:
predicate_uri = MYNS["Foot"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("Foot", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [25]:
predicate_uri = MYNS["shirtNumber"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("shirtNumber", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [26]:
predicate_uri = MYNS["strength"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("strength", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [27]:
predicate_uri = MYNS["weakness"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("weakness", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [28]:
predicate_uri = MYNS["styleofPlay"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("styleofPlay", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [29]:
predicate_uri = MYNS["worldfootball_url"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("worldfootball_url", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [30]:
predicate_uri = MYNS["whoscored_url"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("whoscored_url", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [31]:
predicate_uri = MYNS["Age"]
my_kg.add((predicate_uri, RDF.type, RDF.Property))
my_kg.add((predicate_uri, RDFS.label, Literal("Age", lang = 'en')))

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [32]:
# predicate_uri = MYNS["url"]
# my_kg.add((predicate_uri, RDF.type, RDF.Property))
# my_kg.add((predicate_uri, RDFS.label, Literal("Age", lang = 'en')))

In [33]:
my_kg.serialize('data.ttl', format="turtle") 

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

In [34]:
# SCHEMA.name
# SCHEMA.brithPlace
# SCHEMA.birthDate
# SCHEMA.nationality
# SCHEMA.height    SCHEMA.QuantitativeValue
# SCHEMA.weight    SCHEMA.QuantitativeValue
# MYNS["position"]
# MYNS["footSize"] SCHEMA.QuantitativeValue
# MYNS["footSize"] a SCHEMA.size
# MYNS["shirtNumber"] 
# MYNS["strength"]
# MYNS["weakness"]
# MYNS["styleofPlay"]
# SCHEMA.startDate SCHEMA.Date
# SCHEMA.endDate   SCHEMA.Date
# SCHEMA.member   SCHEMA.SportsTeam


In [35]:
SCHEMA.brithPlace

rdflib.term.URIRef('https://schema.org/brithPlace')

## DATA

In [36]:
player_df.columns

Index(['team_history', 'player_url_x', 'name', 'Born', 'Placeofbirth',
       'Nationality_x', 'Height_x', 'Weight', 'Position(s)', 'Foot',
       'Current_team_info', 'Homepage', 'sizeofshoe', 'birth_block_x',
       'player_id_x', 'match_player_id', 'player_url_y', 'Name',
       'Shirt Number', 'Age', 'Height_y', 'Nationality_y', 'Positions',
       'Strengths', 'Weaknesses', 'Style_of_Play', 'Current Team', 'DOB',
       'birth_block_y', 'player_id_y', 'player_id_tot'],
      dtype='object')

In [37]:
def nulltoNone(x):
    if x == 'nan':
        return "None"
    else:
        return x

In [68]:
for idx, row in player_df.iterrows():     
    player_uri = URIRef(MYNS[row["player_id_tot"]])
    my_kg.add((player_uri, RDF.type, SCHEMA.athlete))
    
    ## url
    if row.player_url_x != 'nan':
        my_kg.add((player_uri, MYNS["worldfootball_url"], Literal(row.player_url_x)))
    if row.player_url_y != 'nan':
        my_kg.add((player_uri, MYNS["whoscored_url"], Literal(row.player_url_y)))
    
    ## name
    if row.name != 'nan':
        my_kg.add((player_uri, SCHEMA.name, Literal(row["name"])))
        my_kg.add((player_uri, RDFS.label, Literal(row["name"])))
    else:
        my_kg.add((player_uri, SCHEMA.name, Literal(row["name"])))
        my_kg.add((player_uri, RDFS.label, Literal(row["name"])))
    
    ## birth date
    try:
        tmp = str(int(row.birth_block_x))
        date = date_parse.parse(tmp[:4] + "-" + tmp[4:6] + "-" + tmp[6:])
        my_kg.add((player_uri, SCHEMA.birthDate, Literal(date,datatype=XSD.date)))
    except:
        try:
            tmp = str(int(row.birth_block_y))
            date = date_parse.parse(tmp[:4] + "-" + tmp[4:6] + "-" + tmp[6:])
            my_kg.add((player_uri, SCHEMA.birthDate, Literal(date,datatype=XSD.date)))
        except:
            a = 1
            
            
    ## place of birth
    if row.Placeofbirth != 'nan':
        tmp = ast.literal_eval(row.Placeofbirth)
        for place in tmp:
            my_kg.add((player_uri, SCHEMA.birthPlace, MYNS[re.sub("[^a-zA-Z]", "_", place)]))
            
            
    ## nationality
    if row.Nationality_x != 'nan':
        tmp = ast.literal_eval(row.Nationality_x)
        my_kg.add((player_uri, SCHEMA.nationality, MYNS[re.sub("[^a-zA-Z]", "_", tmp[0])]))
    elif row.Nationality_y != 'nan':
        my_kg.add((player_uri, SCHEMA.nationality, MYNS[re.sub("[^a-zA-Z]", "_", row.Nationality_y)]))
    
    ## Height
    if row.Height_x != 'nan':
        tmp = row.Height_x.split("'")[1][:3]
        my_kg.add((player_uri, SCHEMA.height,Literal(float(tmp),datatype=XSD.float) ))
    elif row.Height_y != 'nan':
        tmp = row.Height_y[:3]
        my_kg.add((player_uri, SCHEMA.height,Literal(float(tmp),datatype=XSD.float) ))
        
    ## Weight
    if row.Weight != 'nan':
        tmp = row.Weight.split("'")[1].split("kg")[0]
        my_kg.add((player_uri, SCHEMA.height,Literal(tmp,datatype=XSD.float) ))
        
    ## Position
    if row["Position(s)"]!= 'nan':
        for pos in row["Position(s)"]:
            pos = re.sub("[^a-zA-Z]", "_", pos)
            my_kg.add((player_uri, MYNS["position"], MYNS[pos] ))
    elif row.Positions!= 'nan':
        for pos in row.Positions:
            pos = re.sub("[^a-zA-Z]", "_", pos)
            my_kg.add((player_uri, MYNS["position"], MYNS[pos] ))
            
    ## Foot
    if row.Foot != 'nan':
        foot = row.Foot.split("'")[1]
        my_kg.add((player_uri, MYNS["Foot"], Literal(foot)))
        
    ## Age
    try:
        a = int(row.Age)
        my_kg.add((player_uri, MYNS["Age"], Literal(row.Age)))
    except:
        a = 1
        
    ## Current Team
    if row.Current_team_info != 'nan':
        js = row.Current_team_info
        js = re.sub("L'A", "LA", js)
        js = re.sub("l'A", "lA", js)
        js = re.sub("O'H", "OH", js)
        js = re.sub("e'e", "ee", js)
        js = re.sub("d'A", "dA", js)
        js = re.sub("a'a", "aa", js)
        js = re.sub("L'H", "LH", js)
        js = re.sub("d'Á", "dÁ", js)
        js = re.sub("'s ", "s ", js)
        js = re.sub('k"s', "ks", js)
        js = re.sub('d\'E', "dE", js)
        current = json.loads(re.sub("'", "\"", js))
        jl = current
        tmp = jl['period'].split(" - ")
        start_month = tmp[0]
        start_month = date_parse.parse(" ".join([start_month.split("/")[1], start_month.split("/")[0], "1"]))
        end_month = tmp[1]
        end_month = date_parse.parse(" ".join([end_month.split("/")[1], end_month.split("/")[0], "1"]))
        team_name =  jl["team_name"]
        position = jl["position"]
        team_url = jl['team_url']
        shirt_number = jl["shirt_Number"]

        revised_name = re.sub("[^a-zA-Z]", "_", team_name)
        current_team = BNode()
        my_kg.add((current_team, MYNS["memberOf"], MYNS[revised_name]))
        my_kg.add((current_team, MYNS["team_name"], Literal(team_name)))
        if position == "Ass. Manager":
            position = "Associate_Manager"
        my_kg.add((current_team, MYNS["position"], MYNS[position]))
        my_kg.add((current_team, MYNS["shirtnumber"], Literal(shirt_number[0][1:])))
        my_kg.add((current_team, MYNS["startAt"], Literal(start_month)))
        my_kg.add((current_team, MYNS["endAt"], Literal(end_month)))
        
        my_kg.add((player_uri, MYNS["Current_Team"], current_team))
        
        
    ## Team history
    team_history = BNode()
    
    if row.team_history != 'nan':
        js = row.team_history
        js = re.sub("L'A", "LA", js)
        js = re.sub("l'A", "lA", js)
        js = re.sub("O'H", "OH", js)
        js = re.sub("e'e", "ee", js)
        js = re.sub("d'A", "dA", js)
        js = re.sub("a'a", "aa", js)
        js = re.sub("L'H", "LH", js)
        js = re.sub("d'Á", "dÁ", js)
        js = re.sub("'s ", "s ", js)
        histories = json.loads(re.sub("'", "\"", js))
        for history in histories:
            team_history_node = BNode()
            jl = histories[history]
            tmp = jl['period'].split(" - ")
            start_month = tmp[0]
            start_month = date_parse.parse(" ".join([start_month.split("/")[1], start_month.split("/")[0], "1"]))
            end_month = tmp[1]
            end_month = date_parse.parse(" ".join([end_month.split("/")[1], end_month.split("/")[0], "1"]))
            team_name =  jl["team"]
            position = jl["position"]
            team_url = jl['team_url']
            
            revised_name = re.sub("[^a-zA-Z]", "_", team_name)
            
            my_kg.add((team_history_node, MYNS["memberOf"], MYNS[revised_name]))
            my_kg.add((team_history_node, MYNS["team_name"], Literal(team_name)))
            if position == "Ass. Manager":
                position = "Associate_Manager"
            my_kg.add((team_history_node, MYNS["position"], MYNS[position]))
            my_kg.add((team_history_node, MYNS["startAt"], Literal(start_month)))
            my_kg.add((team_history_node, MYNS["endAt"], Literal(end_month)))
            my_kg.add((player_uri, MYNS["past_team"], team_history_node))
#     my_kg.add((player_uri, MYNS["Team_History"], team_history))


    ##Strength
    if row.Strengths != 'nan' and row.Strengths != '[]':
        strength = ast.literal_eval(row.Strengths)
        for i in strength:
            node = BNode()
            my_kg.add((player_uri, MYNS["strength"], node))
            my_kg.add((node, MYNS["strength"], Literal(i)))

    ##Weakness
    if row.Weaknesses != 'nan' and row.Weaknesses != '[]':
        weakness = ast.literal_eval(row.Weaknesses)
        for i in weakness:
            node = BNode()
            my_kg.add((player_uri, MYNS["weakness"], node))
            my_kg.add((node, MYNS["weakness"], Literal(i)))
            
    ##player_df["Style_of_Play"]
    if row.Style_of_Play != 'nan' and row.Style_of_Play != '[]':
        Style_of_Play = ast.literal_eval(row.Style_of_Play)
        for i in Style_of_Play:
            node = BNode()
            my_kg.add((player_uri, MYNS["Style_of_Play"], node))
            my_kg.add((node, MYNS["Style_of_Play"], Literal(i)))
    
    
            
    

In [69]:
# my_kg.serialize('data.ttl', format="turtle") 

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>

## Team

In [41]:
merged_team = pd.read_csv(data_dir + "team_join.csv", index_col = 0)
merged_team

,wiki_team_url,name_x,Fullname,Nickname(s),Founded_x,Ground,Capacity,Owner,Chairman,Manager,...,Opened,id,name_y,world_team_url,Country,Founded_y,stadium,Homepage,name,who_team_url
0,https://en.wikipedia.org/wiki/Wigan_Athletic,Wigan Athletic,WiganAthleticFootballClub,"['TheLatics', 'TheTics']",{'1932'},['DWStadium'],25138,['Phoenix2021Limited'],TalalAlHammad,['LeamRichardson'],...,NaN,0,Wigan Athletic,https://www.worldfootball.net/teams/wigan-athl...,['England'],['01/01/1932'],['DWStadium'],['https//www.wiganathletic.com/'],Wigan,https://www.whoscored.com/Teams/194/Show/Engla...
1,https://en.wikipedia.org/wiki/Tottenham_Hotspur,Tottenham Hotspur,TottenhamHotspurFootballClub,['TheLilywhites'],{'1882-09-05'},['TottenhamHotspurStadium'],62850,"['ENICInternationalLtd.', '(85.55%)']",DanielLevy,NaN,...,NaN,1,Tottenham Hotspur,https://www.worldfootball.net/teams/tottenham-...,['England'],['05/09/1882'],['TottenhamHotspurStadium'],['www.spurs.co.uk'],Tottenham,https://www.whoscored.com/Teams/30/Show/Englan...
2,https://en.wikipedia.org/wiki/Birmingham_City,Birmingham City F.C.,BirminghamCityFootballClub,['Blues'],{'1875'},"[""StAndrew's""]",29409,"['BirminghamSportsHoldings75.00%', 'OrientalRa...",NaN,NaN,...,NaN,2,Birmingham City,https://www.worldfootball.net/teams/birmingham...,['England'],['01/01/1875'],"[""StAndrew'sGround""]",['www.bcfc.com'],Birmingham,https://www.whoscored.com/Teams/157/Show/Engla...
3,https://en.wikipedia.org/wiki/Wolverhampton_Wa...,Wolverhampton Wanderers,WolverhamptonWanderersFootballClub,['WolvesTheWanderers'],{'1877'},['MolineuxStadium'],32050,['FosunInternational'],JeffShi,NaN,...,NaN,3,Wolverhampton Wanderers,https://www.worldfootball.net/teams/wolverhamp...,['England'],['01/01/1877'],['MolineuxStadium'],['https//www.wolves.co.uk/'],Wolverhampton Wanderers,https://www.whoscored.com/Teams/161/Show/Engla...
4,https://en.wikipedia.org/wiki/Sunderland_AFC,Sunderland A.F.C.,SunderlandAssociationFootballClub,['TheBlackCats'],{'1879'},['StadiumofLight'],49000,"['KyrilLouis-Dreyfus', '(majority)']",KyrilLouis-Dreyfus,NaN,...,NaN,4,Sunderland AFC,https://www.worldfootball.net/teams/sunderland...,['England'],['01/01/1879'],['StadiumofLight'],['www.safc.com'],Sunderland,https://www.whoscored.com/Teams/16/Show/Englan...
5,https://en.wikipedia.org/wiki/Liverpool_FC,Liverpool,LiverpoolFootballClub,['TheReds'],{'1892-06-03'},['Anfield'],53394,['FenwaySportsGroup'],TomWerner,['JürgenKlopp'],...,NaN,5,Liverpool FC,https://www.worldfootball.net/teams/liverpool-fc/,['England'],['15/03/1892'],['Anfield'],['www.liverpoolfc.com/'],Liverpool,https://www.whoscored.com/Teams/26/Show/Englan...
6,https://en.wikipedia.org/wiki/Fulham_FC,Fulham,FulhamFootballClub,['TheCottagersTheWhitesTheLilyWhites'],{'1879'},['CravenCottage'],19359,['ShahidKhan'],ShahidKhan,NaN,...,NaN,6,Fulham FC,https://www.worldfootball.net/teams/fulham-fc/,['England'],['01/01/1879'],['CravenCottage'],['www.fulhamfc.com'],Fulham,https://www.whoscored.com/Teams/170/Show/Engla...
7,https://en.wikipedia.org/wiki/Manchester_City,Manchester City,ManchesterCityFootballClub,"['CityCityzens', 'TheCitizensTheSkyBlues']","{'1880', '1894-04-16', '1887'}",['CityofManchesterStadium'],53400,['CityFootballGroup'],KhaldoonAlMubarak,['PepGuardiola'],...,NaN,7,Manchester City,https://www.worldfootball.net/teams/manchester...,['England'],['01/01/1880'],['EtihadStadium'],['https//www.mancity.com/'],Manchester City,https://www.whoscored.com/Teams/167/Show/Engla...
8,https://en.wikipedia.org/wiki/Blackburn_Rovers,Blackburn Rovers,BlackburnRoversFootballClub,"['Rovers', 'TheBlueandWhites', 'TheRiversiders']",{'1875'},['EwoodPark'],31367,"[""Venky'sLtd."", '(99.9%)']",NaN,['TonyMowbray'],...,NaN,8,Blackburn Rovers,https://www.worldfootball.net/teams/blackburn-...,['England'],['05/11/1875'],['EwoodPark'],['https//www.rovers.co.uk/'],Blackburn,https://www.whoscored.com/Teams/158/Show/Engla...
9,https://en.wikipedia.org/wiki/Aston_Villa,Aston Villa,AstonVillaFootballClub,"['TheVilla',

In [42]:
team_url_list = []
for i, j in team_list:
    team_url_list.append(j)

target_name_list = []
for idx, row in merged_team.iterrows():
    for name, url in set(team_list):
        if url == row.world_team_url:
            revised_name = re.sub("[^a-zA-Z]", "_", name)
            target_name_list.append(revised_name) 
merged_team["myns_name"] = target_name_list

In [43]:
merged_team = merged_team.fillna("nan") 

In [44]:
merged_team.columns

Index(['wiki_team_url', 'name_x', 'Fullname', 'Nickname(s)', 'Founded_x',
       'Ground', 'Capacity', 'Owner', 'Chairman', 'Manager', 'League',
       '2020–21', 'Website', 'Homecolours', 'Headcoach', 'Shortname',
       'Coordinates', 'Architect', 'Expanded', 'Maincontractors', 'Renovated',
       'HeadCoach', 'CEO', 'Owner(s)', 'Joint-Chairman', 'Co-chairmen',
       'Owners', 'Stadium', 'Anear-fullcapacity', 'Location', 'Opened', 'id',
       'name_y', 'world_team_url', 'Country', 'Founded_y', 'stadium',
       'Homepage', 'name', 'who_team_url', 'myns_name'],
      dtype='object')

### League

In [45]:
#League list
league_list = [ast.literal_eval(i)[0] for i in set(merged_team["League"])]

In [46]:
league_list

['PremierLeague', 'EFLChampionship', 'LeagueOne', 'EFLLeagueOne']

In [47]:
for league in league_list:
    league_uri = URIRef(MYNS[league])
    my_kg.add((league_uri, RDF.type, MYNS["League"]))
    my_kg.add((league_uri, RDFS.label, Literal(league)))

In [48]:
merged_team.columns

Index(['wiki_team_url', 'name_x', 'Fullname', 'Nickname(s)', 'Founded_x',
       'Ground', 'Capacity', 'Owner', 'Chairman', 'Manager', 'League',
       '2020–21', 'Website', 'Homecolours', 'Headcoach', 'Shortname',
       'Coordinates', 'Architect', 'Expanded', 'Maincontractors', 'Renovated',
       'HeadCoach', 'CEO', 'Owner(s)', 'Joint-Chairman', 'Co-chairmen',
       'Owners', 'Stadium', 'Anear-fullcapacity', 'Location', 'Opened', 'id',
       'name_y', 'world_team_url', 'Country', 'Founded_y', 'stadium',
       'Homepage', 'name', 'who_team_url', 'myns_name'],
      dtype='object')

### Stadium

In [49]:
stadium_list = list(merged_team.stadium.apply(lambda x : re.sub(r"(\w)([A-Z])", r"\1 \2",x[2:-2])))

In [50]:
for cap, stadium in zip(list(merged_team.Capacity), stadium_list):
    stadium_name = re.sub("[^a-zA-Z]", "_", stadium)
    stadium_uri = URIRef(MYNS[stadium_name])
    my_kg.add((stadium_uri, RDF.type, MYNS["Stadium"]))
    my_kg.add((stadium_uri, RDFS.label, Literal(stadium)))
    my_kg.add((stadium_uri, MYNS["capacity"], Literal(int(cap))))

## Team Info

In [51]:
for idx, row in merged_team.iterrows():
    team_uri = MYNS[row.myns_name]
    ## url
    if row.world_team_url != 'nan':
        my_kg.add((team_uri, MYNS["worldfootball_url"], Literal(row.world_team_url)))
    if row.who_team_url != 'nan':
        my_kg.add((team_uri, MYNS["whoscored_url"], Literal(row.who_team_url)))
    if row.wiki_team_url != 'nan':
        my_kg.add((team_uri, MYNS["wikipedia_url"], Literal(row.wiki_team_url)))        
        
    ##team_name
    my_kg.add((team_uri, MYNS["name"], Literal(row.name_x)))
    
    ##Founded
    tmp = row.Founded_y.split("'")[1].split('/')
    if len(tmp) == 3:
        tmp = date_parse.parse(" ".join([tmp[2], tmp[1], tmp[0]]))
    elif len(tmp) == 2:
        tmp = date_parse.parse(" ".join([tmp[1], tmp[0]]))
    elif len(tmp) == 1:
        tmp = date_parse.parse(" ".join([tmp[0]]))
    my_kg.add((team_uri, MYNS['FoundedAt'], Literal(tmp)))
    
    ##Chairman
    if row.Chairman != 'nan':
        tmp = re.sub(r"(\w)([A-Z])", r"\1 \2",row.Chairman)
        my_kg.add((team_uri, MYNS['Chairman'], Literal(tmp)))
        
    ##Owner
    if row.Owner != 'nan':
        tmp = ast.literal_eval(row.Owner)[0]
        tmp = re.sub(r"(\w)([A-Z])", r"\1 \2", tmp)
        my_kg.add((team_uri, MYNS['owner'], Literal(tmp)))
        
    ##League
    if row.League != 'nan':
        league = ast.literal_eval(row.League)[0]
        my_kg.add((team_uri, MYNS["league"], MYNS[league]))
        
    ##Website
    if row.Homepage != 'nan':
        my_kg.add((team_uri, MYNS["Website"], Literal(row.Homepage[2:-2])))
    
    ##Stadium
    if row.stadium != 'nan':
        my_kg.add((team_uri, MYNS["mainStadium"], MYNS[re.sub("[^a-zA-Z]", "_", stadium)]))
    
    

In [52]:
# my_kg.serialize(data_dir + 'data.ttl', format="turtle") 

## Game

In [53]:
game_merged = pd.read_csv(data_dir + "game_join.csv")

### Season

In [54]:
season_list = game_merged["season_world"].drop_duplicates().dropna()

In [55]:
for season in season_list:
    season_name = re.sub("[^a-zA-Z0-9]", "_", season)
    season_uri = URIRef(MYNS[season_name])
    my_kg.add((season_uri, RDF.type, MYNS["season"]))
    my_kg.add((season_uri, RDFS.label, Literal(season_name)))

In [56]:
game_merged.isna().sum()

season_world        347
round               347
data                347
time_world          347
home_team_world     347
                   ... 
BSD                3259
BSA                3259
SBH                3652
SBD                3652
SBA                3652
Length: 116, dtype: int64

In [57]:
def f(x):
    return str(x.home_team_id) + "__" + str(x.away_team_id) +"__" + re.sub("-","_",x.date)
game_merged["identifier"] = game_merged.apply(f, axis = 1)

In [58]:
def f(x):
    if x!="nan":
        return x.split("/")[4]
player_df["who_id"] = player_df["player_url_y"].apply(f)

In [59]:
merged_team2 = pd.read_csv(data_dir + "team_match.csv", index_col = 0)

merged_team2.reset_index(inplace = True)

team_url_list = []
for i, j in team_list:
    team_url_list.append(j)

target_name_list = []
for idx, row in merged_team2.iterrows():
    for name, url in set(team_list):
        if url == row.world_team_url:
            revised_name = re.sub("[^a-zA-Z]", "_", name)
            target_name_list.append(revised_name) 
merged_team2["myns_name"] = target_name_list

In [60]:
merged_team = merged_team.merge(merged_team2, left_on = ['wiki_team_url','world_team_url','who_team_url', 'id', 'myns_name'], right_on = ['wiki_team_url','world_team_url','who_team_url', 'id', 'myns_name'], how = 'outer')


In [61]:
merged_team.loc[38,'name'] = 'brentford'
merged_team.loc[38,'name_x'] = 'brentford'

In [62]:
## structured team name

Lev = NormalizedLevenshtein().similarity

structured_name = []
for idx, row in merged_team.iterrows():
    sim = 0
    str_name = 0
    for name in list(game_merged["HomeTeam"].dropna().drop_duplicates()):
        if Lev(row.name_x, name)>sim:
            sim = Lev(row.name_x, name)
            str_name = name
    
    structured_name.append(str_name)

In [63]:
merged_team['structured_name'] = structured_name

In [64]:
merged_team.loc[36, 'structured_name'] = 'nan'
merged_team.loc[35, 'structured_name'] = 'nan'
merged_team.loc[24, 'structured_name'] = 'QPR'
merged_team.loc[22, 'structured_name'] = 'Hull'
merged_team.loc[20, 'structured_name'] = 'Swansea'
merged_team.loc[11, 'structured_name'] = 'Stoke'
merged_team.loc[3, 'structured_name'] = 'Wolves'

In [65]:
game = game_merged.fillna("nan")
for idx, row in tqdm(game.iterrows()):
    game_uri = URIRef(MYNS[row["identifier"]])
    my_kg.add((game_uri, RDF.type, MYNS["event"]))
    
    ## season
    if row.season_world != "nan":
        my_kg.add((game_uri, MYNS["seasonOf"], MYNS[re.sub("[^a-zA-Z0-9]", "_", row.season_world)]))
    elif row.season_who != "nan":
        my_kg.add((game_uri, MYNS["seasonOf"], MYNS[re.sub("[^a-zA-Z0-9]", "_", row.season_who)]))
        
    ## round
    if row['round'] != "nan":
        my_kg.add((game_uri, MYNS["roundOf"], Literal(int(row["round"]))))
    
    ## score
    if row["score"] != "nan":
        my_kg.add((game_uri, MYNS["scoreOf"], Literal(row["score"])))
    elif row["FTHG"] != "nan":
        score = str(row.FTHG) + ":" + str(row.FTAG)
        my_kg.add((game_uri, MYNS["scoreOf"], Literal(score)))
        
    ##date
    if row["time_world"] != "nan":
        if row["date"] != "nan":
            my_kg.add((game_uri, MYNS["DateTime"], Literal(date_parse.parse(row.date + " " + row.time_world))))
    elif row["time_who"] != "nan":
        if row["date"] != "nan":
            my_kg.add((game_uri, MYNS["DateTime"], Literal(date_parse.parse(row.date + " " + row.time_who))))
    elif row["date"] != "nan":
        my_kg.add((game_uri, MYNS["DateTime"], Literal(date_parse.parse(row.date))))   
        
    ##home_team
    if row["home_team_url_who"] != 'nan':
        home_team_id_who =row.home_team_url_who.split("/")[4]
        home_team_name = merged_team[merged_team['who_team_url'].apply(lambda x : x.split("/")[4]) == home_team_id_who]['myns_name'].values[0]
        my_kg.add((game_uri, MYNS["awayTeam"], MYNS[home_team_name]))
    elif row['home_team_url_world'] != 'nan':
        home_team_name = merged_team[merged_team['world_team_url'] == row["home_team_url_world"]]['myns_name'].values[0]
        my_kg.add((game_uri, MYNS["homeTeam"], MYNS[home_team_name])) 
    elif row["HomeTeam"] != 'nan':
        home_team_name = merged_team[merged_team['structured_name'] == row["HomeTeam"]]['myns_name'].values[0]
        my_kg.add((game_uri, MYNS["homeTeam"], MYNS[home_team_name])) 
     
    if row["away_team_url_who"] != 'nan':
        away_team_id_who =row.away_team_url_who.split("/")[4]
        away_team_name = merged_team[merged_team['who_team_url'].apply(lambda x : x.split("/")[4]) == away_team_id_who]['myns_name'].values[0]
        my_kg.add((game_uri, MYNS["awayTeam"], MYNS[away_team_name]))
    elif row['away_team_url_world'] != 'nan':
        away_team_name = merged_team[merged_team['world_team_url'] == row["away_team_url_world"]]['myns_name'].values[0]
        my_kg.add((game_uri, MYNS["awayTeam"], MYNS[away_team_name])) 
    elif row["AwayTeam"] != 'nan':
        away_team_name = merged_team[merged_team['structured_name'] == row["AwayTeam"]]['myns_name'].values[0]
        my_kg.add((game_uri, MYNS["awayTeam"], MYNS[away_team_name]))
        
        
    if row.home_team_players != "nan":   
        tmp = json.loads(re.sub(r"(\w)\"(\w)", r"\1-\2",re.sub("'", "\"",row.home_team_players)))
#         tmp = json.loads(re.sub(r"(\w)\"(\w)", r"\1-\2",re.sub("'", "\"",row.home_team_players)))
        home_url_list = []
        for i in tmp:
            home_url_list.append(i["player_url"])
            
    if row.away_team_players != "nan":   
        tmp = json.loads(re.sub(r"(\w)\"(\w)", r"\1-\2",re.sub("'", "\"",row.away_team_players)))
        away_url_list = []
        for i in tmp:
            away_url_list.append(i["player_url"])
     
    goal_player_id_list = []
    if row.goal_player_url != "nan" and row.home_team_players != '[]':
        for url in ast.literal_eval(row.goal_player_url):
            try:
                goal_player_id = player_df[player_df["player_url_x"] == url]["player_id_tot"].values[0]
                goal_player_id_list.append(goal_player_id)
            except:
                continue
    goal_counter = Counter(goal_player_id_list)           
            
     
    ## Player rating
    if row["player_rating"] != "nan":
        rating_dict = ast.literal_eval(row["player_rating"])
        for player_key in rating_dict:
            stmt = BNode()
            try:
                player_id = player_df[player_df["who_id"] == player_key].fillna("nan")["player_id_tot"].values[0]
            except:
                continue
            rating = rating_dict[player_key]["rating"]
            shirt_number = rating_dict[player_key]["shirt_number"].split(".")[0]

            my_kg.add((MYNS[player_id], MYNS["participate_in"], stmt))
            if player_id in goal_player_id_list:
                my_kg.add((stmt, MYNS["goal_assist"], Literal(int(goal_counter[player_id]), datatype=XSD.integer)))
            my_kg.add((stmt, MYNS["participate_in"], game_uri))
            my_kg.add((stmt, MYNS["rating"], Literal(rating)))
            my_kg.add((stmt, MYNS["shirt_number"], Literal(shirt_number)))
            
            try:
                player_url = player_df[player_df["who_id"] == player_key].fillna("nan")["player_url_x"].values[0]
            except:
                continue
                
            if player_url in home_url_list:
                my_kg.add((stmt, MYNS["teamAgainst"], MYNS[away_team_name]))
            if player_url in away_url_list:
                my_kg.add((stmt, MYNS["teamAgainst"], MYNS[home_team_name]))
            
            
            

4838it [10:48,  7.46it/s]


In [66]:
my_kg.serialize(data_dir + 'data.ttl', format="turtle") 

<Graph identifier=Ndd76c1df1d4b44e4b54ec48544e9a70e (<class 'rdflib.graph.Graph'>)>